In [12]:
import time
import json
import os
import pandas as pd
import numpy as np
import flwr as fl
import networkx as nx

from sklearn.model_selection import train_test_split

from hydra import initialize, compose
from omegaconf import OmegaConf, DictConfig

from logging import INFO, DEBUG
from flwr.common.logger import log


from src.models.evaluation_metrics import custom_acc_mc, custom_acc_binary

from src.data.dataset_info import datasets

with initialize(version_base=None, config_path="conf/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))

# choosing the dataset
dataset = datasets[0]
print("dataset: {}".format(dataset.name))
# folder_path = "./fl_from_2_datasets_pca/"
# folder_path = "./datasets_pca_federated_new/"
folder_path = "./datasets_bdp/"

learning_rate = 0.0001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

multi_class: false
with_network_features: false
n_clients: 5
n_rounds: 20
config_fit:
  lr: 0.01
  momentum: 0.9
  local_epochs: 1
  batch_size: 256

dataset: cic_ton_iot


In [13]:
dtime = time.strftime("%Y%m%d-%H%M%S")
dtime

'20240815-211133'

In [14]:
clients_paths = [
    folder_path + "test.parquet",
    folder_path + "client_0_pca.parquet",
    folder_path + "client_1_pca.parquet",
    folder_path + "client_2_pca.parquet",
    folder_path + "client_3_pca.parquet",
    folder_path + "client_4_pca.parquet",
    folder_path + "client_5_pca.parquet",
    folder_path + "client_6_pca.parquet",
    folder_path + "client_7_pca.parquet"
]

In [15]:
clients_paths = [
    folder_path + "test.parquet",
    folder_path + "client_0_pca.parquet",
    folder_path + "client_1_pca.parquet",
    folder_path + "client_2_pca.parquet",
    folder_path + "client_3_pca.parquet",
    folder_path + "client_4_pca.parquet",
    folder_path + "client_5_pca.parquet",
    folder_path + "client_6_pca.parquet",
    folder_path + "client_7_pca.parquet"
]
client_columns = []

for client_path in clients_paths:
    df = pd.read_parquet(client_path)
    client_columns.append(set(df.columns))

common_columns = set.intersection(*client_columns)

differences = [columns - common_columns for columns in client_columns]

#for idx, columns in enumerate(client_columns):
 #   print(f"Client {idx} columns: {columns}")

print(f"\nIntersection of columns across all clients: {common_columns}")

for idx, diff in enumerate(differences):
    print(f"Difference in columns for client {idx}: {diff}")


Intersection of columns across all clients: {'Flow Byts/s', 'Pkt Len Min', 'TotLen Bwd Pkts', 'Tot Bwd Pkts', 'ACK Flag Cnt', 'pca_1', 'Fwd Pkts/b Avg', 'Active Mean', 'Timestamp', 'Bwd Pkts/b Avg', 'dst_betweenness', 'Bwd Pkt Len Mean', 'src_betweenness', 'src_pagerank', 'Tot Fwd Pkts', 'Bwd Blk Rate Avg', 'Fwd Blk Rate Avg', 'Idle Mean', 'Flow ID', 'Attack', 'Pkt Len Std', 'Fwd Seg Size Min', 'Fwd PSH Flags', 'Subflow Fwd Byts', 'Bwd Header Len', 'Idle Min', 'Bwd Pkt Len Min', 'src_degree', 'Flow IAT Max', 'Label', 'Pkt Len Var', 'Init Fwd Win Byts', 'RST Flag Cnt', 'Bwd Pkt Len Max', 'Fwd Header Len', 'FIN Flag Cnt', 'Src IP', 'Class', 'Active Std', 'PSH Flag Cnt', 'Bwd Byts/b Avg', 'Dst Port', 'Bwd Pkt Len Std', 'Fwd Pkts/s', 'Fwd URG Flags', 'Fwd Pkt Len Min', 'dst_degree', 'Bwd URG Flags', 'Bwd IAT Tot', 'Fwd Act Data Pkts', 'Fwd IAT Mean', 'Active Min', 'Protocol', 'Down/Up Ratio', 'Idle Max', 'Pkt Size Avg', 'Bwd IAT Max', 'Flow IAT Min', 'CWE Flag Count', 'Fwd Seg Size Avg', 

In [16]:
cn_2 = [
    "dst_global_betweenness",
    "src_global_degree",
    "dst_global_degree",
    "src_mv",
    "src_global_pagerank",
    "dst_global_pagerank",
    "src_global_betweenness",
    "dst_mv"
]

cn_1 = [
    "dst_local_pagerank",
    "src_local_betweenness",
    "src_Comm",
    "src_local_degree",
    "dst_local_betweenness",
    "dst_Comm",
    "dst_local_degree",
    "src_local_pagerank"
]

for i, client_path in enumerate(clients_paths):
    # Determine which set of columns to drop based on the client index
    if i < 5:
        drop_columns = cn_2
    else:
        drop_columns = cn_1
    df = pd.read_parquet(client_path)
    df.drop(columns=drop_columns, errors='ignore', inplace=True)
test = pd.read_parquet(folder_path + "test.parquet")
df.drop(columns=cn_2, errors='ignore', inplace=True)


# Data Loading and Preprocessing

In [17]:
import pickle
# the input dimension of the training set
# input_dim = df.shape[1] - len(drop_columns) - len(weak_columns) - 1  # for the label_column
  
# specifying the number of classes, since it is different from one dataset to another and also if binary or multi-class classification
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2
if cfg.multi_class:
    with open(folder_path + "labels_names.pkl", 'rb') as f:
        labels_names, classes_set = pickle.load(f)
    num_classes = len(classes_set)
    
labels_names = {int(k): v for k, v in labels_names.items()}

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'benign', 'attack'}
==>> num_classes: 2
==>> labels_names: {0: 'benign', 1: 'attack'}


In [18]:
print(folder_path + "test.parquet")

./datasets_bdp/test.parquet


In [20]:
test = pd.read_parquet(folder_path + "test.parquet")
# test = pd.read_parquet("fl_from_2_datasets/preprocessed/test.parquet")
test.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Attack,Class,src_betweenness,dst_betweenness,src_degree,dst_degree,src_pagerank,dst_pagerank,pca_1,pca_2
117477,192.168.1.190-192.168.1.35-80-34692-6,192.168.1.190,80.0,192.168.1.35,34692.0,6.0,2019-04-27 21:12:03,166030.0,4.0,4.0,...,Benign,0,1.667824e-03,9.059850e-05,0.009442,0.000745,0.000300,0.000133,-0.220379,-0.561945
488690,192.168.1.195-192.168.1.79-58678-9197-6,192.168.1.195,58678.0,192.168.1.79,9197.0,6.0,2019-04-27 17:08:13,363.0,2.0,0.0,...,Benign,0,2.104518e-03,8.135874e-06,0.002145,0.000238,0.000316,0.000047,-0.206620,-0.639181
1495319,192.168.1.35-192.168.1.195-42260-80-6,192.168.1.35,42260.0,192.168.1.195,80.0,6.0,2019-04-27 21:10:12,25084.0,5.0,4.0,...,injection,16,9.059850e-05,2.104518e-03,0.000745,0.002145,0.000133,0.000316,0.664609,-0.414151
69616,192.168.1.39-18.194.169.124-58026-80-6,192.168.1.39,58026.0,18.194.169.124,80.0,6.0,2019-04-27 21:33:17,268.0,2.0,0.0,...,xss,21,6.762077e-05,7.539369e-09,0.000655,0.000238,0.000128,0.000042,0.157520,-0.983877
2499459,52.28.231.150-192.168.1.39-80-47584-6,52.28.231.150,80.0,192.168.1.39,47584.0,6.0,2019-04-27 20:03:05,751556.0,7.0,7.0,...,Benign,0,7.539369e-09,6.762077e-05,0.000238,0.000655,0.000042,0.000128,0.206437,-0.981731


In [21]:


if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
test.drop(["pca_1", "pca_2"], axis=1, inplace=True)
if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]

client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
    cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

# Model

In [22]:
from keras import layers, models, Input, regularizers, callbacks, metrics, optimizers, initializers
# from src.models.evaluation_metrics import f1_m

def create_keras_model(input_shape, alpha = learning_rate):
    model = models.Sequential()
    
    model.add(layers.Conv1D(80, kernel_size=3,
                activation="relu", input_shape=(input_shape, 1), kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    # .L1L2(l1=LAMBD_1, l2=LAMBD_2)
    model.add(layers.Conv1D(80, 3, activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    
    # model.add(layers.LSTM(units=80,
    #                         activation='relu',
    #                         kernel_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         recurrent_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         bias_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         return_sequences=False,
    #                         ))
    # model.add(layers.LayerNormalization(axis=1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(80,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))

    if cfg.multi_class:
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    else:
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    
    
    return model


In [23]:
model = create_keras_model(input_dim)
model.summary()

c:\Users\Administrateur\Desktop\centralities-with-fl-nids-main\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 36, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 18, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 18, 80)         │            36 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 16, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 8, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 8, 80)          │            16 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       128,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,173 (801.46 KB)

 Trainable params: 205,173 (801.46 KB)

 Non-trainable params: 0 (0.00 B)

# FL

## FL settings

In [24]:
results_final = {}

results_final["model"] = {}

results_final["baseline"] = {}
results_final["baseline"]["accuracy"] = {}
results_final["baseline"]["f1s"] = {}

results_final["centralities - PCA"] = {}
results_final["centralities - PCA"]["accuracy"] = {}
results_final["centralities - PCA"]["f1s"] = {}

results_final["centralities - DiGraph"] = {}
results_final["centralities - DiGraph"]["accuracy"] = {}
results_final["centralities - DiGraph"]["f1s"] = {}

results_final["centralities - MultiDiGraph"] = {}
results_final["centralities - MultiDiGraph"]["accuracy"] = {}
results_final["centralities - MultiDiGraph"]["f1s"] = {}

In [25]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - baseline"
results["dtime"] = dtime
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - baseline',
 'dtime': '20240815-211133',
 'multi_class': False,
 'learning_rate': 0.0001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 38,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS H

In [26]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/baseline/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [27]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [28]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/baseline/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["baseline"]["accuracy"][server_round] = scores["accuracy"]
        results_final["baseline"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [30]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")
    results
    return metrics
    # total_examples = 0
    # federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    # for num_examples, m in metrics:
    #     for k, v in m.items():
    #         federated_metrics[k] += num_examples * v
    #     total_examples += num_examples
    # return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    # fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.

# strategy.aggregate_fit = 


## FL Simulation

In [31]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-08-15 21:37:28,167 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-15 21:37:33,476	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-15 21:37:36,070 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 12640101580.0, 'memory': 25280203163.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-08-15 21:37:36,072 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-15 21:37:36,074 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 3, 'num_gpus': 0.0}
INFO flwr 2024-08-15 21:37:36,099 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 10 actors
INFO flwr 2024-08-15 21:37:36,101 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-15 21:37:36,102 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.3558 - loss: 1.7025
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step


c:\Users\Administrateur\Desktop\centralities-with-fl-nids-main\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\centralities-with-fl-nids-main\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\centralities-with-fl-nids-main\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa

==>> weighted_average: [(489061, {'accuracy': 0.941207766532898, 'loss': 0.7245796918869019, 'val_accuracy': 0.9493752121925354, 'val_loss': 0.6017386317253113}), (454119, {'accuracy': 0.9954681396484375, 'loss': 0.49481987953186035, 'val_accuracy': 0.9944706559181213, 'val_loss': 0.30321046710014343}), (489061, {'accuracy': 0.9294770956039429, 'loss': 0.7650564908981323, 'val_accuracy': 0.9363279938697815, 'val_loss': 0.6472267508506775}), (489061, {'accuracy': 0.950599193572998, 'loss': 0.7105528116226196, 'val_accuracy': 0.9613919258117676, 'val_loss': 0.5903477668762207}), (489061, {'accuracy': 0.9547234177589417, 'loss': 0.692179262638092, 'val_accuracy': 0.9645203351974487, 'val_loss': 0.5684477090835571}), (423074, {'accuracy': 0.9417926669120789, 'loss': 0.7606189846992493, 'val_accuracy': 0.9537535309791565, 'val_loss': 0.651157557964325}), (454119, {'accuracy': 0.9759380221366882, 'loss': 0.6470673680305481, 'val_accuracy': 0.9866621494293213, 'val_loss': 0.5170385837554932})

RuntimeError: Simulation crashed.

In [ ]:
print(f"==>> history: {history}")
print(f"==>> end of history")

In [ ]:
# creating the directories if they don't exist
if not os.path.isdir('./results'):
    os.mkdir('./results')

# creating the directories if they don't exist
if not os.path.isdir('./results/{}'.format(dtime)):
    os.mkdir('./results/{}'.format(dtime))

# if not os.path.isdir('./results/{}'.format(dataset_name)):
#     os.mkdir('./results/{}'.format(dataset_name))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

filename = ('./results/{}/baseline.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

# FL__FL-PCA

In [ ]:
test = pd.read_parquet(folder_path + "test.parquet")

if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
#test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]

client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
    cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
   # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

In [ ]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - PCA"
results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

In [ ]:
model = create_keras_model(input_dim)
model.summary()

In [ ]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        # log(INFO, f"==>> config: {config}")
        # log(INFO, f"==>> float(config[lr]): {lr}")
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/PCA/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [ ]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [ ]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/PCA/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["centralities - PCA"]["accuracy"][server_round] = scores["accuracy"]
        results_final["centralities - PCA"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [ ]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")

    total_examples = 0
    federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    for num_examples, m in metrics:
        for k, v in m.items():
            federated_metrics[k] += num_examples * v
        total_examples += num_examples
    return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.


In [ ]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_gpus": 0.0,
    },
)

In [ ]:
print(f"==>> history: {history}")
print(f"==>> end of history")

In [ ]:
filename = ('./results/{}/pca.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [ ]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# Centralities - DiGraph

In [ ]:
test = pd.read_parquet(folder_path + "test.parquet")

if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]

client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
    # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

In [ ]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - Centralities - DiGraph"
results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

In [ ]:
model = create_keras_model(input_dim)
model.summary()

In [ ]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        # log(INFO, f"==>> config: {config}")
        # log(INFO, f"==>> float(config[lr]): {lr}")
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/digraph/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [ ]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [ ]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/digraph/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["centralities - DiGraph"]["accuracy"][server_round] = scores["accuracy"]
        results_final["centralities - DiGraph"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [ ]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")

    total_examples = 0
    federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    for num_examples, m in metrics:
        for k, v in m.items():
            federated_metrics[k] += num_examples * v
        total_examples += num_examples
    return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.


In [ ]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_gpus": 0.0,
    },
)

In [ ]:
print(f"==>> history: {history}")
print(f"==>> end of history")

In [ ]:
filename = ('./results/{}/digraph.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [ ]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# Centralities - MultiDiGraph

In [ ]:
# test = pd.read_parquet(folder_path + "test.parquet")

# if cfg.multi_class:
#     test[dataset.label_col] = test[dataset.class_num_col]
    
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# # test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
# test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

# if not cfg.multi_class:
#     test_by_class = {}
#     classes = test[dataset.class_col].unique()
#     for class_value in classes:
#         test_class = test[test[dataset.class_col] == class_value].copy()
#         test_class.drop(dataset.drop_columns, axis=1, inplace=True)
#         test_class.drop(dataset.weak_columns, axis=1, inplace=True)
#         test_class.reset_index(drop=True, inplace=True)

#         test_class_labels = test_class[dataset.label_col].to_numpy()
#         test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

#         test_by_class[class_value] = (test_class, test_class_labels)
    
    
# test.drop(dataset.drop_columns, axis=1, inplace=True)
# test.drop(dataset.weak_columns, axis=1, inplace=True)
# test.reset_index(drop=True, inplace=True)
    
# test_labels = test[dataset.label_col].to_numpy()
# test = test.drop([dataset.label_col], axis=1).to_numpy()
# input_dim = test.shape[1]

# client_data = []
# for client_path in clients_paths:
#     client_data.append(pd.read_parquet(client_path))
    
# for i in range(len(client_data)):
    
#     cdata = client_data[i]
    
#     if cfg.multi_class:
#         cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
#     cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
#     # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
#     cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

#     cdata.drop(dataset.drop_columns, axis=1, inplace=True)
#     cdata.drop(dataset.weak_columns, axis=1, inplace=True)
#     cdata.reset_index(drop=True, inplace=True)
    
#     c_train, c_test = train_test_split(cdata, test_size=0.1)

#     y_train = c_train[dataset.label_col].to_numpy()
#     x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
#     y_test = c_test[dataset.label_col].to_numpy()
#     x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
#     client_data[i] = (x_train, y_train, x_test, y_test)

In [ ]:
# results = {}  # a dictionary that will contain all the options and results of models
# # add all options to the results dictionary, to know what options selected for obtained results
# results["configuration"] = "2dt - Centralities - MultiDiGraph"
# results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
# results["multi_class"] = cfg.multi_class
# results["learning_rate"] = learning_rate
# results["dataset_name"] = dataset.name
# results["num_classes"] = num_classes
# results["labels_names"] = labels_names
# results["input_dim"] = input_dim

# results["scores"] = {}
# results["scores"]["server"] = {}
# results["scores"]["clients"] = {}
# results["scores"]["accuracy"] = {}
# results["scores"]["f1s"] = {}

# if not cfg.multi_class:
#     results["scores"]["test_by_class"] = {}
#     results["scores"]["test_by_class"]["accuracy"] = {}
#     results["scores"]["test_by_class"]["f1s"] = {}
#     for k in test_by_class.keys():
#         results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
#         results["scores"]["test_by_class"]["accuracy"][k] = {}   
#         results["scores"]["test_by_class"]["f1s"][k] = {}    
        
# results

In [ ]:
# model = create_keras_model(input_dim)
# model.summary()

In [ ]:

# class FLClient(fl.client.NumPyClient):
#     def __init__(self, cid, x_train, y_train, x_test, y_test):
#         self.cid = cid
#         self.x_train, self.y_train = x_train, y_train
#         self.x_test, self.y_test = x_test, y_test
#         self.model = create_keras_model(input_shape=input_dim)

#     def get_parameters(self, config):
#         return self.model.get_weights()

#     def set_parameters(self, parameters, config):
#         self.model.set_weights(parameters)

#     def fit(self, parameters, config):
        
#         lr=float(config["lr"])
#         # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
#         self.model = create_keras_model(input_shape=input_dim, alpha=lr)
#         # log(INFO, f"==>> config: {config}")
#         # log(INFO, f"==>> float(config[lr]): {lr}")
#         self.set_parameters(parameters, config)

        
#         logdir = "logs/scalars/{}/multidigraph/client_{}".format(dtime, self.cid)
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         history = self.model.fit(self.x_train, self.y_train,
#                                  epochs=config["local_epochs"],
#                                  batch_size=config["batch_size"],
#                                  validation_data=(self.x_test, self.y_test),
#                                  verbose=0,
#                                  callbacks=[tensorboard_callback])

#         return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters, config)
#         loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
#         return loss, len(self.x_test), {"accuracy": accuracy}


In [ ]:
# def generate_client_fn():
#     def client_fn(cid: str):
#         i = int(cid)
#         return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

#     return client_fn

In [ ]:
# def get_on_fit_config(config: DictConfig):

#     def fit_config_fn(server_round: int):
#         alpha = learning_rate
#         if server_round > 5:
#             alpha = alpha / (1 + 0.5 * server_round)


#         return {
#             "lr": alpha,
#             "local_epochs": config.local_epochs,
#             "batch_size": config.batch_size,
#         }

#     return fit_config_fn


# def get_evaluate_fn(x_test_sever, y_test_server):

#     def evaluate_fn(server_round: int, parameters, config):
#         # eval_model = model
#         eval_model = create_keras_model(input_shape=input_dim)
#         eval_model.set_weights(parameters)

        
#         logdir = "logs/scalars/{}/multidigraph/server".format(dtime) 
#         # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
#                                                   batch_size = cfg.config_fit.batch_size,
#                                                   callbacks=[tensorboard_callback])
        
        
#         y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
#         if cfg.multi_class:
#             y_pred = np.argmax(y_pred, axis=1)
#             scores = custom_acc_mc(y_test_server, y_pred)
#         else:
#             y_pred = np.transpose(y_pred)[0]
#             y_pred = list(
#                 map(lambda x: 0 if x < 0.5 else 1, y_pred))
#             scores = custom_acc_binary(y_test_server, y_pred)
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
#         results_final["centralities - MultiDiGraph"]["accuracy"][server_round] = scores["accuracy"]
#         results_final["centralities - MultiDiGraph"]["f1s"][server_round] = scores["f1s"]
        
#         if not cfg.multi_class:
#             for k in test_by_class.keys():
#                 y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
#                 y_pred_class = np.transpose(y_pred_class)[0]
#                 y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
#                 scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
#                 results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
#                 results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
#         log(INFO, f"==>> scores: {scores}")
        
        
#         return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

#     return evaluate_fn


In [ ]:
# def weighted_average(metrics):
#     print(f"==>> weighted_average: {metrics}")

#     total_examples = 0
#     federated_metrics = {k: 0 for k in metrics[0][1].keys()}
#     for num_examples, m in metrics:
#         for k, v in m.items():
#             federated_metrics[k] += num_examples * v
#         total_examples += num_examples
#     return {k: v / total_examples for k, v in federated_metrics.items()}

# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
#     min_fit_clients=len(client_data),  # number of clients to sample for fit()
#     fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
#     min_evaluate_clients=0,  # number of clients to sample for evaluate()
#     min_available_clients=len(client_data),  # total clients in the simulation
#     fit_metrics_aggregation_fn = weighted_average,
#     # evaluate_metrics_aggregation_fn = weighted_average,
#     on_fit_config_fn=get_on_fit_config(
#         cfg.config_fit
#     ),  # a function to execute to obtain the configuration to send to the clients during fit()
#     evaluate_fn=get_evaluate_fn(test, test_labels),
# )  # a function to run on the server side to evaluate the global model.


In [ ]:
# import multiprocessing
# from math import floor
# history = fl.simulation.start_simulation(
#     client_fn=generate_client_fn(),  # a function that spawns a particular client
#     # num_clients=cfg.n_clients,  # total number of clients
#     num_clients=len(client_data),  # total number of clients
#     config=fl.server.ServerConfig(
#         num_rounds=cfg.n_rounds
#         # num_rounds=5
#     ),  # minimal config for the server loop telling the number of rounds in FL
#     strategy=strategy,  # our strategy of choice
#     client_resources={
#         "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
#         "num_gpus": 0.0,
#     },
# )

In [ ]:
# print(f"==>> history: {history}")
# print(f"==>> end of history")

In [ ]:
# filename = ('./results/{}/multidigraph.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results, cls=NumpyEncoder))
# outfile.close()

In [ ]:
# filename = ('./results/{}/results_final.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
# outfile.close()